# Модуль 3

In [4]:
import os
import random
from ultralytics import YOLO

#### Входе анализа изображений было онаружено что фактически классов 100, от 00 до 99. Причина по которой они были раскиданы по отльным папкам не ясна. Для более правильного обучения модели было принято решение отказаться от такой струкруты дата сета. Подпапки были соеденены воедино.

In [ ]:
def display_project_structure():
    print('preprocessed_images')
    print('    |-- train')
    print('    |   |-- 00')
    print('    |   |-- 01')
    print('    |   |-- 02')
    print('    |   |-- ...')
    print('    |   |-- 99')
    print('    |-- test')
    print('    |   |-- 00')
    print('    |   |-- 01')
    print('    |   |-- 02')
    print('    |   |-- ...')
    print('    |   |-- 99')
    print('    |-- val')
    print('        |-- 00')
    print('        |-- 01')
    print('        |-- 02')
    print('        |-- ...')
    print('    |   |-- 99')

In [17]:
import os
import shutil

def group_images_by_suffix(train_path):
    # Проходим по всем подпапкам в папке train
    for class_folder in os.listdir(train_path):
        class_folder_path = os.path.join(train_path, class_folder)

        # Проверяем, является ли это папкой
        if os.path.isdir(class_folder_path):
            # Извлекаем последние две цифры из имени папки
            class_id = class_folder[-2:]  # последние 2 символа

            # Создаем папку для класса, если она не существует
            target_folder = os.path.join(train_path, class_id)
            os.makedirs(target_folder, exist_ok=True)

            # Перемещаем все файлы в соответствующую папку
            for file_name in os.listdir(class_folder_path):
                file_path = os.path.join(class_folder_path, file_name)

                # Проверяем, является ли это файлом
                if os.path.isfile(file_path):
                    # Определяем путь назначения
                    destination_path = os.path.join(target_folder, file_name)

                    # Если файл с таким именем уже существует, пропускаем его
                    if os.path.exists(destination_path):
                        print(f"Файл {file_name} уже существует в {target_folder}. Пропускаем.")
                        continue  # Пропускаем перемещение этого файла

                    # Перемещаем файл
                    shutil.move(file_path, destination_path)

            # Удаляем пустую папку после перемещения
            try:
                os.rmdir(class_folder_path)
                print(f"Папка {class_folder_path} была удалена.")
            except OSError as e:
                print(f"Ошибка при удалении папки {class_folder_path}: {e}")

#### Применяем функцию group_images_by_suffix

In [ ]:
group_images_by_suffix('/home/c13/main_venv/projects/preprocessed_images/train')

#### Так же применяем функцию group_images_by_suffix к папкам test val

In [ ]:
group_images_by_suffix('/home/c13/main_venv/projects/preprocessed_images/test')
group_images_by_suffix('/home/c13/main_venv/projects/preprocessed_images/val')

#### Создаем метки класов для модели Yolo

In [20]:
def create_yolo_labels(image_folder):
    """
    Создает метки для изображений в формате YOLO в той же папке, где находятся изображения.

    :param image_folder: Путь к корневой папке с изображениями, организованными по классам.
    """
    # Проходим по всем классам (папкам) в указанной папке
    for class_name in os.listdir(image_folder):
        class_path = os.path.join(image_folder, class_name)

        # Проверяем, является ли это папкой
        if os.path.isdir(class_path):
            # Получаем индекс класса
            class_index = int(class_name)  # Предполагаем, что названия папок - это числа от 00 до 99

            # Проходим по всем изображениям в папке класса
            for image_name in os.listdir(class_path):
                if image_name.endswith(('.jpg', '.jpeg', '.png')):  # Проверяем расширение файла
                    # Создаем имя файла для меток
                    label_file_name = os.path.splitext(image_name)[0] + '.txt'
                    label_file_path = os.path.join(class_path, label_file_name)  # Сохраняем в той же папке

                    # Записываем метку в файл
                    with open(label_file_path, 'w') as label_file:
                        # В YOLO формат меток: <class_index> <x_center> <y_center> <width> <height>
                        # Здесь мы просто добавим класс, а координаты оставим пустыми
                        # В реальной задаче вам нужно будет указать координаты объектов
                        label_file.write(f"{class_index} 0.5 0.5 1.0 1.0\n")  # Пример метки

# Пример использования функции
train_folder = '/home/c13/main_venv/projects/preprocessed_images/train'
create_yolo_labels(train_folder)

# Для валидации, вы можете вызвать ту же функцию с другой папкой
val_folder = '/home/c13/main_venv/projects/preprocessed_images/val'
create_yolo_labels(val_folder)

#### Применяем функцию create_yolo_labels для создания меток

In [21]:
train_folder = '/home/c13/main_venv/projects/preprocessed_images/val'
output_labels_folder = '/home/c13/main_venv/projects/yolo_labels/val'
create_yolo_labels(train_folder, output_labels_folder)

#### Определяем параметры для модели Yolo

In [45]:
params = {
    'data': '/home/c13/main_venv/projects/data.yaml',  # Путь к файлу с данными
    'epochs': 1,  # Количество эпох
    'batch': 16,  # Размер батча
    'imgsz': 320,  # Размер изображений
    'patience': 0,  # Количество эпох без улучшения для ранней остановки
    'cache': True,  # Кэширование данных для ускорения
}


## Создаем функцию для обучеиня моделей. 

In [43]:
import time
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_iris
import subprocess
import os

def train_model(model_type, **kwargs):
    if model_type == 'logistic_regression':
        model = LogisticRegression(**kwargs)
        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time

        y_val_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_val_pred)
        roc_auc = roc_auc_score(y_val, model.predict_proba(X_val), multi_class='ovr')
        f1 = f1_score(y_val, y_val_pred, average='macro')

    elif model_type == 'random_forest':
        model = RandomForestClassifier(**kwargs)
        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time

        y_val_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_val_pred)
        roc_auc = roc_auc_score(y_val, model.predict_proba(X_val), multi_class='ovr')
        f1 = f1_score(y_val, y_val_pred, average='macro')

    elif model_type == 'yolov5n':
        # Start training YOLOv5
        model = YOLO("yolov5n.yaml")
        start_time = time.time()
        results = model.train(**kwargs)  # Pass parameters to train method
        training_time = time.time() - start_time

        print("Training completed in {:.2f} seconds.".format(training_time))
        return results

    else:
        raise ValueError("Unsupported model type. Choose from 'logistic_regression', 'random_forest', or 'yolov5n'.")

In [ ]:
results = train_model('yolov5n', **params)

Ultralytics 8.3.70 🚀 Python-3.11.2 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
engine/trainer: task=detect, mode=train, model=yolov5n.yaml, data=/home/c13/main_venv/projects/data.yaml, epochs=1, time=None, patience=0, batch=16, imgsz=320, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

train: Scanning /home/c13/main_venv/projects/preprocessed_images/train/00.cache... 23004 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23004/23004 [00:00<?, ?it/s]

train: 9.9GB RAM required to cache images with 50% safety margin but only 2.5/15.4GB available, not caching images ⚠️



val: Scanning /home/c13/main_venv/projects/preprocessed_images/val/00.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.6GB RAM): 100%|██████████| 2000/2000 [00:00<00:00, 7217.94it/s] 


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=9.6e-05, momentum=0.9) with parameter groups 69 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to runs/detect/train7
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      2.803      5.495      3.918         57        320:  81%|████████▏ | 1170/1438 [27:05<06:01,  1.35s/it]

# Отчет
##### Создана новая структура датасета
##### Созданы метки классов для модели Yolo
##### Написана функция для обучения нескольких моделей